
A DVD rental company wants to figure out how many days a customer will rent a DVD for based on some features. We try out some regression models to help predict the number of days a customer will rent a DVD for. We wants a model which yields a MSE of 3 or less on a test set. 

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [53]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Lasso then Linear Regression
from sklearn.linear_model import Lasso, LinearRegression
# Random Forest with Parameters search
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


data = pd.read_csv('rental_info.csv')

# Data Preprocessing

In [54]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [55]:
# Rental length 
data["rental_length"] = pd.to_datetime(data["return_date"]) - pd.to_datetime(data["rental_date"])

data["rental_length_days"] = data["rental_length"].dt.days


# Dummy variables for special features

data["deleted_scenes"] =   np.where(data["special_features"].str.contains("Deleted Scenes"), 1, 0)

data["behind_the_scenes"] =  np.where(data["special_features"].str.contains("Behind the Scenes"), 1, 0)

data.columns

Index(['rental_date', 'return_date', 'amount', 'release_year', 'rental_rate',
       'length', 'replacement_cost', 'special_features', 'NC-17', 'PG',
       'PG-13', 'R', 'amount_2', 'length_2', 'rental_rate_2', 'rental_length',
       'rental_length_days', 'deleted_scenes', 'behind_the_scenes'],
      dtype='object')

In [56]:
# Choose relevant features
features = [ 'amount', 'release_year', 'rental_rate', 'length', 'replacement_cost', 'amount_2', 'length_2', 'rental_rate_2', 'deleted_scenes', 'behind_the_scenes']
# 'NC-17', 'PG', 'PG-13', 'R'

target = 'rental_length_days'

X = data[features]
y = data[target]

# Training models 

- LASSO selection followed by Linear Regression
- Random Forest Regression with Parameters search

In [57]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=9)


In [58]:
# LASSO
lasso = Lasso(alpha=0.2, random_state=9) 

lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# choose columns with positive coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]


# Apply Linear Regression Model
lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X_lasso_train, y_train)

# Evaluate
y_test_pred = lin_reg.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)
mse_lin_reg_lasso

4.812297241276244

In [59]:
# Random forest

# Parameters space
params = {'n_estimators': np.arange(1,51,2), 'max_depth':np.arange(1,7,1)}

# Model
rf = RandomForestRegressor()

# Search parameters
search = RandomizedSearchCV(rf, param_distributions=params, cv=5, random_state=9)
search.fit(X_train, y_train)
best_params = search.best_params_
print(best_params)

# Train with those param
rf = RandomForestRegressor(n_estimators=best_params["n_estimators"], 
                           max_depth=best_params["max_depth"], 
                           random_state=9)
rf.fit(X_train,y_train)

# Evaluate
rf_pred = rf.predict(X_test)
mse_random_forest= mean_squared_error(y_test, rf_pred)
mse_random_forest

{'n_estimators': 45, 'max_depth': 6}


2.4577611138663444

The Random Forest model fulfill the requirements. 

In [60]:
best_model = rf
best_mse = mse_random_forest